# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Vocabulary

---

# Observations et environnement
---

In [1]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [2]:
import importlib
import pandas as pd
import seaborn as sns

## Articles & Labels

In [3]:
import patat.db.label_db
label_db = patat.db.label_db.LabelDb()
label_db.df()

,url,label,value,owner,type,url_h
0,https://www.alnas.fr/actualite/communaute/lara...,infox,0.000000,recueil,obs,ef05018ec0f5d637bceb4cef7004e8ee5fe48f6f
1,https://www.alnas.fr/actualite/communaute/lara...,signe,0.000000,recueil,obs,ef05018ec0f5d637bceb4cef7004e8ee5fe48f6f
2,https://www.alnas.fr/actualite/communaute/lara...,entites_nommees,1.000000,recueil,obs,ef05018ec0f5d637bceb4cef7004e8ee5fe48f6f
3,https://www.alnas.fr/actualite/communaute/lara...,entites_coherentes,1.000000,recueil,obs,ef05018ec0f5d637bceb4cef7004e8ee5fe48f6f
4,https://www.alnas.fr/actualite/communaute/lara...,ouverture_esprit,0.000000,recueil,obs,ef05018ec0f5d637bceb4cef7004e8ee5fe48f6f
...,...,...,...,...,...,...
266583,https://www.profession-gendarme.com/zelensky-e...,infox,0.556761,220930-tf_lr.pp,pred,e346761c2a99d892b26c9388480e0ee6ad6b303b
266584,https://www.profession-gendarme.com/zelensky-l...,infox,0.164531,220930-tf_lr.pp,pred,77525354644316eadab53960efa8f5fd028c7f67
266585,https://www.profession-gendarme.com/zelensky-m...,infox,0.673791,220930-tf_lr.pp,pred,fc6a074da5c02032bc9fc3f35f1044bc4029042c
266586,https://www.profession-gendarme.com/zero-mort-...,infox,0.103192,220930-tf_lr.pp,pred,e4f0a8961d79c9a5f66c80d7aa4196cb679af2b0


In [4]:
import patat.db.article_db
importlib.reload(patat.db.article_db)
article_db = patat.db.article_db.ArticleDb()
article_db.df()

/Users/fm/Desktop/Work/Patat/patat/db/article_db.py:37: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


,url,title,article,author,tag,abstract,site,date_iso,url_h,c_count
0,https://www.alnas.fr/actualite/en-vrac/les-emi...,Les Émirats arabes unis et l’Arabie saoudite f...,Plus des trois quarts des importations japonai...,Par Sarah -,"['Actualités', 'En vrac']",NaN,www.alnas.fr,2023-01-03T00:00:00,d9726057bcbe0182c5bec27f5cc2e1c73594a361,NaN
1,https://www.anguillesousroche.com/changement-c...,Article d’opinion du NY Times : « S’accoupler ...,« Nous serons plus nombreux à pouvoir monter d...,NaN,['Changement climatique'],NaN,www.anguillesousroche.com,2023-01-04T00:00:00,7dce1f2479231f6e9606c69da67444b634c2d32d,NaN
2,https://www.bvoltaire.fr/prediction-pour-2023-...,Prédiction pour 2023 : ça va bien se passer… j...,"Le 31 décembre dernier, Emmanuel Macron était,...",Frédéric Lassez,['Articles\nPolitique\nVoeux Macron'],NaN,www.bvoltaire.fr,2023-01-03T00:00:00,b11055abd5b8f6c385a79096ae29d90f8a0348ff,NaN
3,https://lesdeqodeurs.fr/le-premier-ministre-or...,Le Premier ministre Orbán affirme que le scand...,Article original datant du 16/12/22 par RMX.ne...,NaN,"['Andrea Cozzolino', 'Bruxelles', 'corruption'...",NaN,lesdeqodeurs.fr,2022-12-19T00:00:00,790fa6da6e1d9151e0ad6ce08627f738ad1e864c,NaN
4,https://lesobservateurs.ch/2022/12/18/herault-...,Hérault : 300 musulmans armés de kalachnikovs ...,"« La horde sauvage » nouvelle version, sans la...",Francis Gruzelle,[],NaN,lesobservateurs.ch,2022-12-18T00:00:00,2b314df4e0710050ae0441d6423177362127c784,NaN
...,...,...,...,...,...,...,...,...,...,...
86822,https://www.lelibrepenseur.org/une-histoire-vi...,Une histoire vieille comme le monde !,Certains veulent des preuves ! Ils ont raison....,Par\nAdmin,NaN,NaN,www.lelibrepenseur.org,2010-05-04T00:00:00,8c7a6ecf46d47da1912884a0c0a2837c69f7ad19,7851.0
86823,https://www.lelibrepenseur.org/actus/,Actus,"Dernière vidéo !\nIl était temps, enfin ! Je v...",Par\nuser,NaN,NaN,www.lelibrepenseur.org,2007-07-07T00:00:00,d4d5a37a8ff545664bea75664c8d8695e70d0941,2173.0
86824,https://www.lelibrepenseur.org/affiche-des-dec...,Affiche des décès et malaises de sportifs de h...,Un nombre inquiétant de sportifs de haut nivea...,NaN,NaN,NaN,www.lelibrepenseur.org,NaN,a175027896cf79e76dfad6409694fafacf1919b3,2193.0
86825,https://www.lelibrepenseur.org/communaute/,Communauté,Notre Réseau Telegram et ses Groupes Secondair...,NaN,NaN,NaN,www.lelibrepenseur.org,NaN,6ba257c7b15dd68cadf7dc5d6d8c584db0f1792e,2926.0


# Experience
---

## Textes

In [5]:
df_labels = label_db.df()

In [6]:
df_infox = df_labels[(df_labels['label']=='infox') & (df_labels['type'] == 'obs')].copy()

In [7]:
df_infox = df_infox[['url','value']]

In [8]:
df_infox = df_infox.rename(columns={'value': 'infox'})

In [9]:
df_infox = df_infox.merge(article_db.df()[['url','title','article']].copy())

In [10]:
df_infox['text'] = df_infox['title'] + '\n' + df_infox['article'] 

In [11]:
df_infox.to_csv('data/tmp/221219-InfoxTexts',index=False)

In [12]:
def is_afp(author):
    if type(author)==str:
        return 'afp' in author.lower()
    else:
        return False

In [13]:
df_articles = article_db.df()

In [14]:
df_francesoir = df_articles[df_articles['site']=='www.francesoir.fr']

In [15]:
df_afp = df_francesoir[df_francesoir['author'].apply(is_afp)][['url','title','article']].copy()

In [16]:
df_afp['text'] = df_afp['title'] + '\n' + df_afp['article'] 

In [17]:
df_afp['infox']=0.0

In [18]:
df_afp.to_csv('data/tmp/221219-AfpTexts',index=False)

### Preprocessor

In [19]:
import patat.util.text

preprocessor = patat.util.text.preprocess


### Tokenizer

In [20]:
from patat.ml.lex_analyser import LexAnalyser

lex = LexAnalyser()

tokenizer = lex.get_lemmas_from_text

### Vocabulary

In [ ]:
df_texts = pd.concat([df_infox,df_afp])

In [73]:
texts = df_texts['text']

In [96]:
values = df_texts['infox'].astype(int)

In [97]:
df_words = lex.get_df_words(texts, values, preprocessor, tokenizer)

/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [98]:
df_words

,count_0,count_1,count
de,115224,24646,139870
la,63847,13219,77066
le,49789,10182,59971
et,39056,10379,49435
les,38357,10599,48956
...,...,...,...
pharmacol,0,3,3
pharmacologique,0,6,6
pharmacologistes,0,1,1
pharmacologue,0,1,1


### Rare Words

In [99]:
for occ_rare in [1,2,3,4,5,6,7]:
    df_words['rare_'+str(occ_rare)]=(df_words['count_0']<=occ_rare)&(df_words['count_1']<=occ_rare)

In [100]:
df_words

,count_0,count_1,count,rare_1,rare_2,rare_3,rare_4,rare_5,rare_6,rare_7
de,115224,24646,139870,False,False,False,False,False,False,False
la,63847,13219,77066,False,False,False,False,False,False,False
le,49789,10182,59971,False,False,False,False,False,False,False
et,39056,10379,49435,False,False,False,False,False,False,False
les,38357,10599,48956,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
pharmacol,0,3,3,False,False,True,True,True,True,True
pharmacologique,0,6,6,False,False,False,False,False,True,True
pharmacologistes,0,1,1,True,True,True,True,True,True,True
pharmacologue,0,1,1,True,True,True,True,True,True,True


### Common words

In [101]:
for common_size in [50,80,120,200,500]:
    common_0 = df_words.sort_values('count_0',ascending=False).head(common_size).index
    common_1 = df_words.sort_values('count_1',ascending=False).head(common_size).index
    df_words['common_'+str(common_size)]=(df_words.index.isin(common_0))&(df_words.index.isin(common_1))

In [102]:
df_words

,count_0,count_1,count,rare_1,rare_2,rare_3,rare_4,rare_5,rare_6,rare_7,common_50,common_80,common_120,common_200,common_500
de,115224,24646,139870,False,False,False,False,False,False,False,True,True,True,True,True
la,63847,13219,77066,False,False,False,False,False,False,False,True,True,True,True,True
le,49789,10182,59971,False,False,False,False,False,False,False,True,True,True,True,True
et,39056,10379,49435,False,False,False,False,False,False,False,True,True,True,True,True
les,38357,10599,48956,False,False,False,False,False,False,False,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pharmacol,0,3,3,False,False,True,True,True,True,True,False,False,False,False,False
pharmacologique,0,6,6,False,False,False,False,False,True,True,False,False,False,False,False
pharmacologistes,0,1,1,True,True,True,True,True,True,True,False,False,False,False,False
pharmacologue,0,1,1,True,True,True,True,True,True,True,False,False,False,False,False


### Vocab & Ignore words

In [ ]:
ignore_words_small = list(df_words[df_words['rare_1']].index) + list(df_words[df_words['common_80']].index)

In [ ]:
ignore_words_medium = list(df_words[df_words['rare_2']].index) + list(df_words[df_words['common_80']].index)

In [ ]:
ignore_words_large = list(df_words[df_words['rare_5']].index) + list(df_words[df_words['common_200']].index)

# Sauvegarde des résultats
---

In [ ]:
df_words.to_csv('data/prod/221125-InfoxWords.csv')

# Conclusions
---

- Vocabulaire infox OK

# Bricolages
---

In [25]:
df_texts

,url,infox,title,article,text
0,https://www.alnas.fr/actualite/communaute/lara...,0.0,L’Arabie saoudite organise des célébrations d'...,L’Arabie saoudite a organisé un événement pour...,L’Arabie saoudite organise des célébrations d'...
1,https://www.alnas.fr/actualite/en-vrac/quel-ar...,0.0,« Quel art ! » : un étudiant en droit invente ...,Une professeure de droit a partagé sur Twitter...,« Quel art ! » : un étudiant en droit invente ...
2,https://www.alnas.fr/actualite/politique/laust...,0.0,L’Australie ne reconnaît plus Jérusalem comme ...,Le gouvernement israélien a qualifié la positi...,L’Australie ne reconnaît plus Jérusalem comme ...
3,https://www.alnas.fr/actualite/communaute/nant...,0.0,Nantes : une mère musulmane meurt poignardée e...,NANTES – Une enquête pour « homicide volontair...,Nantes : une mère musulmane meurt poignardée e...
4,https://www.alnas.fr/actualite/en-vrac/la-fran...,0.0,La France condamnée pour avoir placé un enfant...,La Cour européenne des droits de l’Homme (CEDH...,La France condamnée pour avoir placé un enfant...
...,...,...,...,...,...
54983,https://www.francesoir.fr/lifestyle-gastronomi...,0.0,La deuxième vie des coquilles d'huîtres,"Les huîtres ont une deuxième vie, du moins leu...",La deuxième vie des coquilles d'huîtres\nLes h...
54987,https://www.francesoir.fr/lifestyle-gastronomi...,0.0,La truffe se fait rare,Victime du réchauffement climatique et d'une m...,La truffe se fait rare\nVictime du réchauffeme...
54999,https://www.francesoir.fr/lifestyle-gastronomi...,0.0,Rungis prépare un Noël opulent malgré les atte...,Malgré les attentats de Paris et la récente ép...,Rungis prépare un Noël opulent malgré les atte...
55009,https://www.francesoir.fr/lifestyle-gastronomi...,0.0,Gastronomie: le Franco-Suisse Benoît Violier m...,Le restaurant du chef cuisinier franco-suisse ...,Gastronomie: le Franco-Suisse Benoît Violier m...


In [44]:
df_texts=df_texts.reset_index()

In [73]:
texts = df_texts['text']

In [74]:
values = df_texts['infox']

In [76]:
vectorizer = CountVectorizer(lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer,token_pattern='')
count_matrix = vectorizer.fit_transform(texts)

/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [77]:
count_array = count_matrix.toarray()
df_count = pd.DataFrame(data=count_array,columns = vectorizer.get_feature_names_out())

In [78]:
df_words = pd.DataFrame()
for value in values.unique():
    df_words['count_'+str(value)] = pd.DataFrame(df_count[values==value].sum(),columns=['count_0']).sort_values('count_0',ascending=False)
df_words['count']=df_count.sum(axis=0)

df_words.shape

(54545, 3)

In [72]:
df_words

,count_0.0,count_1.0,count
de,115224,24646,139870
la,63847,13219,77066
le,49789,10182,59971
et,39056,10379,49435
les,38357,10599,48956
...,...,...,...
pharmacol,0,3,3
pharmacologique,0,6,6
pharmacologistes,0,1,1
pharmacologue,0,1,1


In [90]:
importlib.reload(patat.ml.lex_analyser)

<module 'patat.ml.lex_analyser' from '/Users/fm/Desktop/Work/Patat/patat/ml/lex_analyser.py'>

In [91]:
lex = patat.ml.lex_analyser.LexAnalyser()

In [92]:
lex.get_df_words(texts, values, preprocessor, tokenizer)

/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,count_0.0,count_1.0,count
de,115224,24646,139870
la,63847,13219,77066
le,49789,10182,59971
et,39056,10379,49435
les,38357,10599,48956
...,...,...,...
pharmacol,0,3,3
pharmacologique,0,6,6
pharmacologistes,0,1,1
pharmacologue,0,1,1


In [82]:
import typing

In [83]:
typing.reveal_type(tokenizer)

Runtime type is 'method'


<bound method LexAnalyser.get_lemmas_from_text of <patat.ml.lex_analyser.LexAnalyser object at 0x2b44d37d0>>

In [104]:
values.unique()

array([0, 1])

In [106]:
pd.MultiIndex.from_product([['value_count'],[0,1]])

MultiIndex([('value_count', 0),
            ('value_count', 1)],
           )